<a href="https://colab.research.google.com/github/paucaroscanoa/ApiBookAuthor/blob/master/Caso_de_Estudio_5_1_Identificando_nuevos_genes_del_autismo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Caso de Estudio 5.1 - Identificando nuevos genes del autismo

Configuración del *notebook*:

Sincronice su cuenta de Google. Para ello, siga el link que aparece en la salida de la siguiente celda una vez ejecutada. Copie el código que le aparece en pantalla e introdúzcalo en la salida de la celda. Una vez vea el mensaje: `Google Drive sincronizado con éxito!` puede continuar ejecutando el resto de celdas.

In [1]:
from google.colab import auth
auth.authenticate_user()

from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
data_drop = drive.CreateFile({'id':'1Iin0h_JR8qiJl8UmJfa7cq-YZ4ou5n2Q'})
data_drop.GetContentFile('BIOGRID.txt')
data_drop = drive.CreateFile({'id':'10FxN4lVk8-dsbdmVLEWEwoKI1wSFAqKs'})
data_drop.GetContentFile('gene-id-table.txt')
data_drop = drive.CreateFile({'id':'1yWjhKnw2knYtLeeX5mzFrJ-qrtEQgAOx'})
data_drop.GetContentFile('gene-score.csv')
data_drop = drive.CreateFile({'id':'1P4MPEtbrQuIUHraBy3nxCKieGFHjK4HG'})
data_drop.GetContentFile('steiner_tree.R')

print('Google Drive sincronizado con éxito!')

Google Drive sincronizado con éxito!


In [2]:
!pip install rpy2==3.5.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.7/201.7 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rpy2: filename=rpy2-3.5.1-cp310-cp310-linux_x86_64.whl size=314958 sha256=151848f9eefb8e773222d25b9ab8239233787a4c1bc6b6c43f94afe2b197c914
  Stored in directory: /root/.cache/pip/wheels/73/a6/ff/4e75dd1ce1cfa2b9a670cbccf6a1e41c553199e9b25f05d953
Successfully built rpy2
  Attempting uninstall: rpy2
    Found existing installation: rpy2 3.4.2
    Uninstalling rpy2-3.4.2:
      Successfully uninstalled rpy2-3.4.2


In [3]:
%load_ext rpy2.ipython

Instalar e importar librerías: (ignorar resultados a no ser que no se muestre la frase: `Librerías instaladas con éxito!`)

In [4]:
%%R
install.packages('igraph', dependencies = TRUE)
cat('\n¡Librerías instaladas con éxito!')

(as ‘lib’ is unspecified)


































	‘/tmp/RtmpAJMwsB/downloaded_packages’




¡Librerías instaladas con éxito!

In [5]:
%%R
install.packages('graph')

(as ‘lib’ is unspecified)



In [6]:
%%R
library(igraph)
cat('\n¡Librerías importadas con éxito!')

Attaching package: ‘igraph’



    decompose, spectrum



    union





¡Librerías importadas con éxito!

## Datos


### Red de interacciones proteína-proteína

Cargar la base de datos BIOGRID.

In [ ]:
%%R
biogrid <- read.delim("BIOGRID.txt",stringsAsFactors = F)

Ver los diferentes componentes de la base de datos.

In [ ]:
%%R
names(biogrid)

Adjuntar la base de datos al workspace y asignar las interacciones entre proteinas a la variable `HSnet`.

In [ ]:
%%R
attach(biogrid)
HSnet <- graph.data.frame(data.frame(Entrez.Gene.Interactor.A,Entrez.Gene.Interactor.B),directed=F)

También podemos ver que forma toma dicha red de interacciones:

In [ ]:
%%R
graph.data.frame(data.frame(Entrez.Gene.Interactor.A,Entrez.Gene.Interactor.B),directed=F)

La siguiente línea le permitirá trazar la red, pero le llevará algo de tiempo. El resultado tendrá mejor calidad si lo ejecuta en un entorno nativo de R (como por ejemplo RStudio).

In [ ]:
%%R
plot(HSnet)

Ahora podemos obtener la matriz de adyacencia de la red, $A$. Como la matriz es muy extensa, podemos visualizar la primera sección de 15x15 celdas:


In [ ]:
%%R
A <- get.adjacency(HSnet)
A[1:15,1:15]

In [ ]:
%%R
#lo siguiente es FALSE si el grafo no es simple.
is.simple(HSnet)

Observamos que existen aristas múltiples en la red, así que vamos a transformarla en una red simple:

In [ ]:
%%R
#eliminar aristas múltiples y autobucles
HSnet <- simplify(HSnet, remove.multiple = TRUE, remove.loops = TRUE, edge.attr.comb = getIgraphOpt("edge.attr.comb"))

is.simple(HSnet)

Observamos que ya sólo hay aristas únicas:

In [ ]:
%%R
A <- get.adjacency(HSnet)
A[1:15,1:15]

In [ ]:
%%R
plot(HSnet)

Para esta aplicación, se eliminan los nodos de grado elevado; normalmente estos nodos representan genes constitutivos que se necesitan para mantener una célula con vida, pero no suelen ser específicos de una enfermedad concreta.

In [ ]:
%%R
overly.attached.proteins <- which(degree(HSnet)>1000)
overly.attached.proteins

In [ ]:
%%R
HSnet <- delete.vertices(HSnet, overly.attached.proteins)

#lo siguiente es TRUE si el grafo está conectado.
is.connected(HSnet)

### Genes que causan autismo

En primer lugar procedemos a leer la tabla `gene-id`:

In [ ]:
%%R
gene.table <- read.delim("gene-id-table.txt")
names(gene.table)

En segundo lugar, también hemos de leer la tabla de puntuaciones para los genes relacionados con el autismo (`gene-scores`):

In [ ]:
%%R
gene.score<-read.csv("gene-score.csv",stringsAsFactors=F)
attach(gene.score)
names(gene.score)

In [ ]:
%%R
#mostrando las puntuaciones
unique(Score)

Ahora podemos identificar aquellos genes con puntuaciones significativas:

In [ ]:
%%R
signif.scores <- c("1S","1","2S","2","3S","3")
signif.genes <- Gene.Symbol[which(Score %in% signif.scores)]
signif.EIDs <- gene.table[which(gene.table[,1] %in% signif.genes),2]

Finalmente, podemos usar la red de interacciones `HSnet` que hemos creado anteriormente, para determinar los genes que están presentes en la red, y que se sabe que causan autismo:

In [ ]:
%%R
geneEIDs <- as.numeric(V(HSnet)$name)
HSnetN <- HSnet
V(HSnetN)$name <- 1:length(V(HSnet))
signif.ids <- which(geneEIDs %in% signif.EIDs)
length(signif.ids)

## Desarrollar un interactoma del autismo

Importamos el script para calcular el `steiner_tree.R`, es decir, la subred de menor longitud que conecta cierto conjunto de nodos.

In [ ]:
%%R
source('steiner_tree.R')

Identificar el *Steiner Tree* y cronometrar el tiempo que se tarda en ejecutar dicha función.

In [ ]:
%%R
system.time(HS.stree <- steiner_tree(terminals=signif.ids, graph=HSnetN))

Mostrar el solapamiento entre significancia de autismo y vértices en el *Steiner Tree*

In [ ]:
%%R
length(intersect(signif.ids,V(HS.stree)$name))
labels<-gene.table[as.numeric(V(HS.stree)$name),1]
labels<-as.character(labels)

Identificar los genes que tienen puntuaciones significativas y asignarles el color rojo

In [ ]:
%%R
colors<-rep("skyblue",length(V(HS.stree)))
colors[which(as.numeric(V(HS.stree)$name) %in% signif.ids)] = "red"
#Asignar colores a los vértices del árbol
V(HS.stree)$color = colors

Exportar el nuevo gráfico a un archivo `.pdf`. Puede acceder al mismo en la barra lateral.

In [ ]:
%%R
pdf("ASD_interactome.pdf",width=12, height=12)
system.time(plot(HS.stree,vertex.label=labels,vertex.size=5,vertex.label.cex=0.8))
dev.off()

## Análisis y propiedades del interactoma del autismo

Para anlizar las propiedades del interactoma del autismoy compararlas con interactomas generados aleatoriamente, usaremos la librería `sna`:

In [ ]:
%%R
install.packages('sna')
library(sna, quietly=TRUE)

En primer lugar, creamos una función que calcule las puntuaciones de conectividad de la red. En este caso, las puntuaciones son los diámetros de la red y la distancia geodésica media entre dos nodos cualesquiera:

In [ ]:
%%R
c.scores<-function(graph){
    n <- length(V(graph))
    sp <- shortest.paths(graph)
    neighbors <- sum(sp==1)/2
    neighbors2 <- sum(sp==2)/2
    return(c(2*neighbors/(n*(n-1)),2*neighbors2/(n*(n-1))))
}

clus <- clusters(HSnetN,mode=c("weak"))
connected.ids <- which(clus$membership==1)
length(connected.ids)

Generamos N subredes de forma aleatoria. Esto puede llevar cierto tiempo si N es grande.

In [ ]:
%%R
N<-50
strees<-list(N)
effs<-numeric(N)
nei<-numeric(N)
nei2<-numeric(N)
for (i in 1:N){
    new.ids<-sample(x=connected.ids,size=length(signif.ids))
    strees[[i]] <- steiner_tree(terminals=new.ids,graph=HSnetN)
    effs[i]<-efficiency(get.adjacency(strees[[i]],sparse=F))
    cs<-c.scores(strees[[i]])
    nei[i]<-cs[1]
    nei2[i]<-cs[2]
}

Podemos mostrar las puntuaciones de eficiencia y conectividad para cada uno de los grafos aleatorios:

In [ ]:
%%R
effs

In [ ]:
%%R
nei

In [ ]:
%%R
nei2

Finalmente, mostramos las puntuaciones de eficiencia y conectividad para el interactoma del autismo


In [ ]:
%%R
efficiency(get.adjacency(HS.stree,sparse=F))

In [ ]:
%%R
c.scores(HS.stree)

## Identificar nuevos genes candidatos

# Nueva sección

Calcular la puntuación de centralidad de intermediación para cada nodo

In [ ]:
%%R
betweeness_centrality_scores = igraph::betweenness(HS.stree)

Identificar aquellos que no sabemos aún si son significativos


In [ ]:
%%R
significant_centrality = c()
count = 0
for (i in 1:length(betweeness_centrality_scores)){
    if (!(as.numeric(names(betweeness_centrality_scores[i])) %in% signif.ids)) {
        significant_centrality = c(significant_centrality,betweeness_centrality_scores[i])
    }
}

Ordenar y mostrar el número de nodos significativos

In [ ]:
%%R
significant_centrality = sort(significant_centrality, decreasing=TRUE)
length(significant_centrality)